In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u

from gammapy.data import DataStore

In [ ]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1")

data_store.info()

`DataStore` contains two tables. The main one is the HDU table. It contains the adresses of all elements contained in all observations.

In [ ]:
data_store.hdu_table[:10]

The other table is the observation summary. It is optional but frequently present. It provides informations (metadata) on the observations present in the `DataStore`.

In [ ]:
data_store.obs_table[:2]

It offers some convenience functions to perform observation selection.

The most common one allows you to select observationd around a given direction.

In [ ]:
position = SkyCoord(ra=83.63, dec=22.01, unit="deg", frame="icrs")
filtered_obs_table = data_store.obs_table.select_sky_circle(position, 3*u.deg)

print(filtered_obs_table)

Now you can extract the identifiers (ObsID) of the selected observations.

In [ ]:
filtered_obs_table['OBS_ID'].data

**Exercice :** Compute the total observation time of observations taken at less than 2° from the SNR RX J1713-3946 position in the HESS DL3 DR1.

### Extracting observations

`Observations` is a sequence (i.e. a list) of `Observation` objects.

In [ ]:
observations = data_store.get_observations(filtered_obs_table['OBS_ID'])

In [ ]:
print(observations)

One can access observations by their index in the list, or simply loop over all observations.

`Observation` contains metadata on the observation.

In [ ]:
obs = observations[1]
print(obs.meta.obs_info)

In [ ]:
obs.peek()

### Events

Gamma-ray like events (i.e. at DL3) are stored in an `EventList` which contains an `astropy.table.Table`

In [ ]:
print(obs.events)

In [ ]:
obs.events.select_offset([0,2.5]*u.deg).peek()

**Exercice**: how many events are there in ObsID 23523 within a circular ON region of 0.3 deg from the Crab?

*Hint : Look for a region selection tool in the API documentation of EventList.*

**Advanced exercice**: measure OFF counts in a symetrical region w.r.t. pointing direction. Compute the excess counts in the ON and the associated Li and Ma significance.

*Hints: Compute the offset and position angle of the ON region w.r.t. `EventList.pointing_radec` and use `SkyCoord.directional_offset_by` to find the OFF region.
Then use CountsStatistics as in https://docs.gammapy.org/1.2/user-guide/stats/index.html#wstat-counts-statistic*

### IRFs : effective area

Let's now look at the IRFs stored in the observation object. 

The effective area IRF is a set of tabulated values at given coordinates in true 3D space 

In [ ]:
print(obs.aeff)

Let's have a view on the IRF.

In [ ]:
obs.aeff.peek()

The IRF also behaves like a function (it interpolates over the tabulated values).

In [ ]:
obs.aeff.evaluate(energy_true=10*u.TeV, offset=1.2*u.deg)

**Exercice**: plot the effective area offset dependence at 500 GeV, 1 and 10 TeV.

*Hint: look for the existing plot functions in the API.*

**Advanced exercice**: Re-implement the plotting yourself.

### Evolution of the observation threshold during an observation night

Because a target is moving in the sky, its zenith angle will vary during the night. Because the energy threshold of a Cherenkov increase with the zenith angle, the latter will vary during the night.  

GADF effective area can contain an effective low energy threshold for the observation. On the HESS DL3 DR1 you can access it with:

In [ ]:
obs.aeff.meta['LO_THRES']

**Exercice** : Extract observations of PKS 2155-304 taken on the night of July 29th to 30th 2006 in the HESS DL3 DR1 datastore. How does the observation threshold evolve as a function of observation time during the night? 

How does it compare to the zenith angle of the target as a function of time?

*Hint: IRCS coordinate is (329.717, -30.226). Use astropy.time.Time. Also note that `Observation.tmid` is a shortcut to the mid observation time of the `Observation`.*
*See also [astropy's doc on altitude of celestial objects](https://docs.astropy.org/en/latest/generated/examples/coordinates/plot_obs-planning.html#determining-and-plotting-the-altitude-azimuth-of-a-celestial-object)*

### EDISP

Provides the energy resolution as a function of position and true energy. It gives the probability to measure a given energy for a given true energy. It is stored as a function of migra = energy/energy_true

In [ ]:
print(obs.edisp)

In [ ]:
obs.edisp.peek()

For a given set of measurement, we use the `EDispKernel` with gives probability to measure a given energy for a given true energy.

In [ ]:
kernel = obs.edisp.to_edisp_kernel(offset=0.5*u.deg)

In [ ]:
kernel.plot_matrix()

### PSF

It gives the probability to observe a photon as given distance from its true direction (the `rad` quantity) as a function of true energy and sourc eposition in the FoV.

In [ ]:
print(obs.psf)

One can compute containment radius as a measurement of the breadth of the PSF

In [ ]:
obs.psf.peek()

PSF is a probability. The PSF radius typically decreases with energy.

In [ ]:
obs.psf.plot_psf_vs_rad(offset=[0.5]*u.deg, energy_true=[0.5, 1, 10]*u.TeV)

### BKG

In [ ]:
print(obs.bkg)

In [ ]:
obs.bkg.peek()

### CTA IRFS

You can also load IRFs without the full `DataStore` with `load_irf_dict_from_file`

In [ ]:
from gammapy.irf import load_irf_dict_from_file
irf_filename = (
        "$GAMMAPY_DATA/cta-caldb/Prod5-South-20deg-AverageAz-14MSTs37SSTs.180000s-v0.1.fits.gz"
)
irfs = load_irf_dict_from_file(irf_filename)
print(irfs)

#### AEFF

In [ ]:
irfs['aeff'].plot()

#### PSF

In [ ]:
irfs["psf"].plot_containment_radius_vs_energy(
    offset=[1] * u.deg, fraction=[0.68, 0.8, 0.95]
)
plt.show()

#### BKG

In [ ]:
irfs["bkg"].plot_at_energy(
    ["50 GeV", "500 GeV", "1 TeV", "3 TeV", "10 TeV", "100 TeV"]
)
plt.show()

**Exercice**: compare IRFs of CTAO North and South sites.

*Hint: you can find the North site IRF here: $GAMMAPY_DATA/cta-caldb/Prod5-North-20deg-AverageAz-4LSTs09MSTs.180000s-v0.1.fits.gz*